<img style="float: left;;" src='Figures/alinco.png' /></a>
# <center> <font color= #000047> Módulo 3: Algoritmos de Aprendizaje por Refuerzo: Q-Learning y SARSA

* En este notebook vamos a resolver un problema con Aprendizaje por refuerzo usando los algoritmos del Q-Learning y SARSA-Learning. El problema que queremos resolver es el de encontrar el camino que nos suponga una mayor recompensa (el más corto) desde un estado inicial $[0,0]$ hasta el estado final $[4,4]$, pudiendo realizar 4 tipos de acciones:


<img src="./Figures/007_RL.png" style="width: 300px;"/>



* Para resolver este problema vamos a realizar lo siguiente:
<span></span><br>
    1. [Definición del entorno](#M1)
<span></span><br>
    2. [Implementación de un algoritmo de toma aleatoria de acciones](#M2)
<span></span><br>
    3. [Ejecución: Entorno - Agente](#M3)
<span></span><br>
    4. [Q-Learner: Implementación y Ejecución](#M4)
<span></span><br>
    5. [SARSA-Learner: Implementación y Ejecución](#M5)
<span></span><br>
    6. [Estrategias a corto y largo plazo](#M5)


<hr>


## Definición del entorno






In [2]:
import numpy as np
r = np.array([[0.0, 0.0, 0.0, 0.0],
                        [0.0, 0.0, 0.0, 0.0],
                        [0.0, 0.0, 0.0, 0.0],
                        [0.0, 0.0, 0.0, 0.0]])
r

array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]])

In [3]:
len(r)

4

In [5]:
len(r[0])

4

In [ ]:
# importamos las librerias necesarias
import pandas as pd
import numpy as np

# Formato de los decimales en Pandas y la semilla del Random
pd.options.display.float_format = '{:,.2f}'.format
np.random.seed(23)


class Environment(object):
    def __init__(self, action_penalty=-1.0):
        """
        Clase que representa y controla en entorno
        :param action_penalty:    Factor de descuento del Reward por acción tomada
        """
        self.action_penalty = action_penalty
        #Accciones que puede tomar el agente
        self.actions = {'Arriba': [0,-1],
                        'Abajo': [0, 1],
                        'Izquierda': [-1,0],
                        'Derecha': [1, 0]}
        self.state = [0,0]
        self.final_state = [3,3]
        self.total_reward = 0.0
        self.actions_done = []
        self.rewards = [[0.0, 0.0, 0.0, 0.0],
                        [0.0, 0.0, 0.0, 0.0],
                        [0.0, 0.0, 0.0, 0.0],
                        [0.0, 0.0, 0.0, 0.0]]


    def reset(self):
        """
        Método que reinicia las variables del entorno y devuelve es estado inicial
        :return:    state
        """
        self.total_reward=0.0
        self.state = [0,0]
        self.actions_done = []
        
        return self.state

    def step(self, action):
        """
        Método que ejecuta una acción determinada del conjunto de acciones {Arriba, Abajo, Izquierda, Derecha}
        para guiar al agente en el entorno.
        :param action:    Acción a ejecutar
        :return:          (state, reward, is_final_state)
        """
        self.__apply_action(action)    #realizamos la acción para el cambio del destado
        self.actions_done.append(self.state[:]) #Guardar el paso (acción realizada)
        is_final_state = np.array_equal(self.state, self.final_state)
        reward = self.reward[self.state[0]][self.state[1]] + self.action_penalty #Se calcula la recompensa por la acción que se tome
        self.total_reward += reward
        
        return self.state, reward, is_final_state

    def __apply_action(self, action):
        """
        Método que calcula el nuevo estado a partir de la acción a ejecutar
        :param action:    Acción a ejecutar
        """
        self.state[0] += self.actions[action][0]  # Estado siguien S' para coordenada en x
        self.state[1] += self.actions[action][1]  # Estado siguien S' para coordenada en y
        
        #Si nos salimos del tablero por arriba o por abajo, nos quedamos en la misma posición que estabamos
        if self.state[0]<0:
            self.state[0] = 0
        elif self.state[0]>len(self.rewards) -1:
            self.state[0] -= 1
        
        #Si nos salimos del tablero por arriba o por abajo, nos quedamos en la misma posición que estabamos
        if self.state[1]<0:
            self.state[1] = 0
        elif self.state[1]>len(self.rewards[0]) -1:
            self.state[1] -= 1
        
        
    def print_path_episode(self):
        """
        Método que imprime por pantalla el camino seguido por el agente
        :return: 
        """
        path = [['-' for _ in range(len(self.rewards))] for _ in range(len(self.rewards[0]))]
        path[0][0] = '0'
        for index, step in enumerate(self.actions_done):
            path[step[0]][step[1]] = str(index + 1)
        
        print(pd.DataFrame(data= np.array([np.array(xi) for xi in path]), 
                                         index = [f"x{str(i)}" for in in range(len(path))], 
                                                 columns = [f"y{str(i)}" for i in range(len(path[0]))]))
        



## Implementación de un algoritmo de toma aleatoria de acciones




In [6]:
class Learner(object):

    def __init__(self, environment, learning_rate=0.1, discount_factor=0.1, ratio_exploration=0.05):
        """
        Clase que implementa un algoritmo de aprendiza por refuerzo
        Esta clase implementa un algoritmo de selección aleatoria de acciones
        :param environment:         Entorno en el que tomar las acciones
        :param learning_rate:       Factor de aprendizaje
        :param discount_factor:     Factor de descuento (0=Estrategia a corto plazo, 1=Estrategia a largo plazo)
        :param ratio_exploration:   Ratio de exploración
        """
        self.enviroment = environment
        self.q_table = [[[0.0 for _ in self.enviroment.actions]
                        for _ in range(len(self.enviroment.rewards))]
                       for _ in range(len(self.enviroment.rewards[0]))]
        self.learning_rate = learning_rate
        self.discount_factor = discount_factor
        self.ratio_exploration = ratio_exploration
        

    @property
    def name(self):
        return 'random'

    def get_next_action(self, state):
        """
        Método que selecciona la siguiente acción a tomar:
            Aleatoria -> si el ratio de exploración es inferior al umbral
            Mejor Acción -> si el ratio de exploración es superior al umbral
        :param state:   Estado del agente
        :return:        next_action
        """
        
        if np.random.uniform() < self.ratio_exploration:
            #seleccionamos una acción al azar
            next_action = np.random.choice(list(self.enviroment.actions))
        else:
            #seleccionamos la acción que nos da mayor valor. si hay empate seleccionamos al azar
            idx_action = np.random.choice(np.flatnonzero(
                self.q_table[state[0]][state[1]] == np.array(self.q_table[state[0]][state[1]]).max()
            ))
            nex_action = list(self.enviroment.actions)[idx_action]
        
        return next_action
    

    def update(self, **kwargs):
        """
        Actualiza la Q-Table
        :param kwargs: 
        """
        pass

    def print_q_table(self):
        """
        Método que imprime por pantalla la Q-Table
        """
        q_table = []
        
        for x, row in enumerate(self.q_table):
            for y, col in enumerate(row):
                q = deepcopy(col)
                q.insert(0,f'x{x},y{y}')
                q_table.append(q)
        
        print(pd.DataFrame(data = q_table, columns = ['Estado', 'Arriba', 'Izquierda', 'Derecha']).to_string(index=False))
        

    def print_best_actions_states(self):
        """
        Método que imprime por pantalla la mejor opción a realizar en cada uno de los estados
        """
        best = [[list(self.enviroment.actions)[np.argmax(col)] for col in row] for row in self.q_table]
        
        print(pd.DataFrame(data=np.array([np.array(xi) for xi in best]),
                           index = [f'x{str(i)}' for i in range(len(best))],
                          columns = [f'y{str(i)}' for i in range(len(best[0]))]))
        
        
    def print_best_values_states(self):
        """
        Método que imprime por pantalla el valor de la mejor opción a realizar en cada uno de los estados
        """
        best = [[max(vi) for vi in row] for row in self.q_table]
        
        print(pd.DataFrame(data=np.array([np.array(xi) for xi in best]),
                           index = [f'x{str(i)}' for i in range(len(best))],
                          columns = [f'y{str(i)}' for i in range(len(best[0]))]))
        


## Ejecución: Entorno - Agente




In [ ]:
from copy import deepcopy


def run_agent(learner=Learner, num_episodes=10, learning_rate=0.1, discount_factor=0.1, ratio_exploration=0.05,
              verbose=False):
    """
    Método que ejecuta el proceso de aprendizaje del agente en un entorno
    :param learner:              Algoritmo de Aprendizaje
    :param num_episodes:         Número de veces que se ejecuta (o aprende) el agente en el entorno
    :param learning_rate:        Factor de Aprendizaje
    :param discount_factor:      Factor de descuento (0=Estrategia a corto plazo, 1=Estrategia a largo plazo)
    :param ratio_exploration:    Ratio de exploración
    :param verbose:              Boolean, si queremos o no imprimir por pantalla información del proceso
    :return:                     (episodes_list, best_episode)
    """

    # Instanciamos el entorno
   

    # Instanciamos el método de aprendizaje
   
    # Variables para guardar la información de los episodios
   
        # Guardamos el mejor episodio
        
            # Imprimimos la información de los episodios
            
    return None, None


def print_process_info(episodes_list, best_episode, print_best_episode_info=True,
                       print_q_table=True, print_best_values_states=True,
                       print_best_actions_states=True, print_steps=True, print_path=True):
    """
    Método que imprime por pantalla los resultados de la ejecución
    """
    


#### Política Aleatoria

<hr>


## Q-Learner: Implementación y Ejecución


* Recordemos el Pseudocódigo del Algoritmo:


<img src="./Figures/013_qlearning.png" style="width: 600px;"/>


In [ ]:
class QLearner(Learner):

    @property
    def name(self):
        return 'QLearner'

    def update(self, environment, old_state, action_taken, reward_action_taken, new_state, is_final_state, **kwargs):
        """
        Método que implementa el Algoritmo de aprendizaje del Q-Learning
        :param environment:           Entorno en el que tomar las acciones
        :param old_state:             Estado actual
        :param action_taken:          Acción a realizar
        :param reward_action_taken:   Recompensa obtenida por la acción tomada
        :param new_state:             Nuevo estado al que se mueve el agente
        :param is_final_state:        Boolean. Devuelve True si el agente llega al estado final; si no, False
        :param kwargs: 
        """
        # Obtenemos el identificador de la acción
        

        # Obtenemos el valor de la acción tomada
        

#### Estrategia a corto plazo

#### Estrategia a largo plazo

<hr>


## SARSA-Learner: Implementación y Ejecución


* Recordemos el Pseudocódigo del Algoritmo:


<img src="./Figures/014_sarsa.png" style="width: 600px;"/>

In [ ]:
class SARSALearner(Learner):

    @property
    def name(self):
        return 'SARSA'

    def update(self, environment, old_state, action_taken, reward_action_taken, new_state, new_action, is_final_state):
        """
        Método que implementa el algoritmo de aprendizaje SARSA
        :param environment:           Entorno en el que tomar las acciones
        :param old_state:             Estado actual
        :param action_taken:          Acción a realizar
        :param reward_action_taken:   Recompensa obtenida por la acción tomada
        :param new_state:             Nuevo estado al que se mueve el agente 
        :param new_action:            Acción a tomar en el nuevo estado
        :param is_final_state:        Boolean. Devuelve True si el agente llega al estado final; si no, False 
        """
        # Obtenemos el identificador de la acción
        

        # Obtenemos el valor de la acción tomada
        


#### Estrategia a corto plazo

#### Estrategia a largo plazo

<hr>


## Estrategias a corto y largo plazo


### Q-Learning

### SARSA